In [19]:
import pyodbc
from loguru import logger
from pathlib import Path
SERVER = 'FRAMGNB107'
DATABASE = 'windmanager_france_test'

tables_path = Path('DATABASES/france_172074') / 'TABLES'
references =  tables_path / '01_REFERENCES'
entities = tables_path / '02_ENTITIES'
relationships = tables_path / '03_RELATIONSHIPS'
look_ups = tables_path / '04_LOOK_UPS'
foreign_keys = tables_path / '05_FOREIGN_KEYS'


try:
    with pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={SERVER};DATABASE={DATABASE};'
        f'Trusted_Connection=yes;TrustServerCertificate=yes;'
    ).cursor() as cur:
        logger.success("Database connection established")
        ### REFERENCES
        cur.execute((references / 'company_roles.sql').read_text())
        logger.info('company_roles created')
        cur.execute((references / 'farm_types.sql').read_text())
        logger.info('farm_types created')
        cur.execute((references / 'person_roles.sql').read_text())
        logger.info('person_roles created')

        ### ENTITIES
        cur.execute((entities / 'companies.sql').read_text())
        logger.info('companies created')
        cur.execute((entities / 'employees.sql').read_text())
        logger.info('employees created')
        cur.execute((entities / 'farms.sql').read_text())
        logger.info('farms created')
        cur.execute((entities / 'ice_detection_systems.sql').read_text())
        logger.info('ice_detection_systems created')
        cur.execute((entities / 'persons.sql').read_text())
        logger.info('persons created')
        cur.execute((entities / 'substations.sql').read_text())
        logger.info('substations created')
        cur.execute((entities / 'wind_turbine_generators.sql').read_text())
        logger.info('wind_turbine_generators created')

        ### RELATIONSHIPS
        cur.execute((relationships / 'farm_company_roles.sql').read_text())
        logger.info('farm_company_roles created')
        cur.execute((relationships / 'farm_referents.sql').read_text())
        logger.info('farm_referents created')

        ### LOOK_UPS
        cur.execute((look_ups / 'farm_actual_performances.sql').read_text())
        logger.info('farm_actual_performances created')
        cur.execute((look_ups / 'farm_administrations.sql').read_text())
        logger.info('farm_administrations created')
        cur.execute((look_ups / 'farm_electrical_delegations.sql').read_text())
        logger.info('farm_electrical_delegations created')
        cur.execute((look_ups / 'farm_environmental_installations.sql').read_text())
        logger.info('farm_environmental_installations created')
        cur.execute((look_ups / 'farm_financial_guarantees.sql').read_text())
        logger.info('farm_financial_guarantees created')
        cur.execute((look_ups / 'farm_ice_detection_systems.sql').read_text())
        logger.info('farm_ice_detection_systems created')
        cur.execute((look_ups / 'farm_legal_auditors.sql').read_text())
        logger.info('farm_legal_auditors created')
        cur.execute((look_ups / 'farm_locations.sql').read_text())
        logger.info('farm_locations created')
        cur.execute((look_ups / 'farm_om_contracts.sql').read_text())
        logger.info('farm_om_contracts created')
        cur.execute((look_ups / 'farm_statuses.sql').read_text())
        logger.info('farm_statuses created')
        cur.execute((look_ups / 'farm_substation_details.sql').read_text())
        logger.info('farm_substation_details created')
        cur.execute((look_ups / 'farm_target_performances.sql').read_text())
        logger.info('farm_target_performances created')
        cur.execute((look_ups / 'farm_tcma_contracts.sql').read_text())
        logger.info('farm_tcma_contracts created')
        cur.execute((look_ups / 'wind_farm_turbine_details.sql').read_text())
        logger.info('wind_farm_turbine_details created')
    logger.success('All tables created')
except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    logger.error(f"Database connection or query failed: {sqlstate} - {ex}")
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")


2025-10-07 16:46:08.399 | SUCCESS  | __main__:<module>:21 - Database connection established


2025-10-07 16:46:08.516 | INFO     | __main__:<module>:24 - company_roles created
2025-10-07 16:46:08.555 | INFO     | __main__:<module>:26 - farm_types created
2025-10-07 16:46:08.583 | INFO     | __main__:<module>:28 - person_roles created
2025-10-07 16:46:08.624 | INFO     | __main__:<module>:32 - companies created
2025-10-07 16:46:08.663 | INFO     | __main__:<module>:34 - employees created
2025-10-07 16:46:08.731 | INFO     | __main__:<module>:36 - farms created
2025-10-07 16:46:08.782 | INFO     | __main__:<module>:38 - ice_detection_systems created
2025-10-07 16:46:08.821 | INFO     | __main__:<module>:40 - persons created
2025-10-07 16:46:08.850 | INFO     | __main__:<module>:42 - substations created
2025-10-07 16:46:08.885 | INFO     | __main__:<module>:44 - wind_turbine_generators created
2025-10-07 16:46:08.915 | INFO     | __main__:<module>:48 - farm_company_roles created
2025-10-07 16:46:08.939 | INFO     | __main__:<module>:50 - farm_referents created
2025-10-07 16:46:08.

In [8]:
import pyodbc
from loguru import logger

# Config
SERVER = 'ZEUS1'
DATABASE = 'france_172074'
USERNAME = 'user_breau'
PASSWORD = 'rEzz?-A6hCc99Gvs'

# 1. Check ODBC drivers
logger.info("Available drivers:")
for driver in pyodbc.drivers():
    logger.info(f"  - {driver}")

# 2. Test connection
logger.info("\nTesting connection...")
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={SERVER};DATABASE={DATABASE};'
        f'UID={USERNAME};PWD={PASSWORD};'
        f'TrustServerCertificate=yes;',
        timeout=10
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT DB_NAME(), COUNT(*) FROM INFORMATION_SCHEMA.TABLES")
    db, tables = cursor.fetchone()
    
    logger.success(f"✓ Connected to {db}")
    logger.success(f"✓ Found {tables} tables")
    
    conn.close()
    
except Exception as e:
    logger.error(f"✗ Failed: {e}")

2025-10-06 12:09:20.003 | INFO     | __main__:<module>:11 - Available drivers:
2025-10-06 12:09:20.017 | INFO     | __main__:<module>:13 -   - SQL Server
2025-10-06 12:09:20.020 | INFO     | __main__:<module>:13 -   - ODBC Driver 17 for SQL Server
2025-10-06 12:09:20.025 | INFO     | __main__:<module>:13 -   - Microsoft Access Driver (*.mdb, *.accdb)
2025-10-06 12:09:20.027 | INFO     | __main__:<module>:13 -   - Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
2025-10-06 12:09:20.029 | INFO     | __main__:<module>:13 -   - Microsoft Access Text Driver (*.txt, *.csv)
2025-10-06 12:09:20.033 | INFO     | __main__:<module>:13 -   - Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
2025-10-06 12:09:20.036 | INFO     | __main__:<module>:16 - 
Testing connection...
2025-10-06 12:09:20.203 | ERROR    | __main__:<module>:36 - ✗ Failed: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'user_breau'. (18456) (SQLDriverConnect); [28000] [Micro